In [ ]:
# Imports and global variables

import random

suits = ('Hearts','Diamonds','Spades','Clubs')
ranks = ('Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack',
         'Queen','King','Ace')
values = {'Two':2,'Three':3,'Four':4,'Five':5,'Six':6,'Seven':7,'Eight':8,
          'Nine':9,'Ten':10,'Jack':10,'Queen':10,'King':10,'Ace':11}
playing = True

In [ ]:
# Card Class: suit and rank

class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return self.rank + " of " + self.suit

In [ ]:
# Deck Class: random list of 52 card objects

class Deck:
    
    def __init__(self):
        self.deck = []  
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))
    
    def __str__(self):
        deck_comp = ''
        for card in self.deck:
            deck_comp += '\n' + card.__str__()
        return 'The deck has: ' + deck_comp

    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        single_card = self.deck.pop()
        return single_card

In [ ]:
# TEST: deck class
test_deck = Deck()
test_deck.shuffle()
print(test_deck)

In [ ]:
# Hand class: calculate value of cards in hand, adjust ace value

class Hand:
    
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0
    
    def add_card(self,card):
        self.cards.append(card)
        self.value += values[card.rank]
        if card.rank == 'Ace':
            self.aces += 1
        
    def adjust_for_ace(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [ ]:
# TEST: hand class
test_deck = Deck()
test_deck.shuffle()
test_player = Hand()
test_player.add_card(test_deck.deal())
test_player.add_card(test_deck.deal())
test_player.value

for card in test_player.cards:
    print(card)

In [ ]:
# Chips class: player's chips, bets, winnings

class Chips:
    
    def __init__(self,total=100):
        self.total = total  
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

In [ ]:
def take_bet(chips):
    '''
    INPUT: number of chips
    OUTPUT: if number of chips is invalid
    '''
    while True:
        try:
            chips.bet = int(input('Enter how many chips you want to bet: '))
        except ValueError:
            print('Bet must be an integer')
        else:
            if chips.bet > chips.total:
                print("Bet can't exceed {}".format(chips.total))
            else:
                break

In [ ]:
def hit(deck,hand):
    '''
    INPUT: deck, hand
    OUTPUT: take a hit/deal card to hand
    '''
    hand.add_card(deck.deal())
    hand.adjust_for_ace()

In [ ]:
def hit_or_stand(deck,hand):
    '''
    INPUT: deck, hand, player input
    OUTPUT: hit or stand using global var
    '''
    global playing  
    
    while True:
        x = input("Hit or Stand? (H/S) ")
        
        if x[0].lower() == 'h':
            print("Player hits.")
            hit(deck,hand)  
        elif x[0].lower() == 's':
            print("Player stands. Dealer's turn.")
            playing = False
        else:
            print("Please enter H/S.")
            continue
        break

In [ ]:
def show_some(player,dealer):
    '''
    OUTPUT: All of player's cards, all but first of dealer's cards
    '''
    print("\nDealer's Hand: ")
    print("<hidden>")
    print(" " + dealer.cards[1].__str__())
    print("\nPlayer's Hand:", *player.cards, sep = '\n')

In [ ]:
def show_all(player,dealer):
    '''
    OUTPUT: All of player's and dealer's cards
    '''
    print("\nDealer's Hand:", *dealer.cards, sep = '\n')
    print("Dealer's Hand =", dealer.value)
    print("\nPlayer's Hand:", *player.cards, sep = '\n')
    print("Player's Hand =", player.value)

In [ ]:
# End of game scenarios

def player_busts(player,dealer,chips):
    print("Player busts!")
    chips.lose_bet()

def player_wins(player,dealer,chips):
    print("Player wins!")
    chips.win_bet()

def dealer_busts(player,dealer,chips):
    print("Dealer busts!")
    chips.win_bet()
    
def dealer_wins(player,dealer,chips):
    print("Dealer wins!")
    chips.lose_bet()
    
def push(player,dealer):
    print("Dealer and Player tie! The hand is a push.")

Run the Game

In [ ]:
#loop to continuously run game until break
while True:
    
    print('Blackjack - Rules:\n1. Get as close to 21 without going over.\n\
    2. Dealer hits until they reach 17. \n\
    3. Aces can be counted as 1 or 11.')
    
    # create and shuffle deck
    deck = Deck()
    deck.shuffle()
    
    # deal to player
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    # deal to dealer
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # first action
    player_chips = Chips()     
    take_bet(player_chips)
    show_some(player_hand,dealer_hand)
    
    while playing:
        
        # player's turn
        hit_or_stand(deck,player_hand) 
        show_some(player_hand,dealer_hand)  
        
        if player_hand.value > 21:
            player_busts(player_hand,dealer_hand,player_chips)
            break
    
    # if player doesn't bust on turn
    if player_hand.value <= 21:
        
        while dealer_hand.value < 17:
            hit(deck,dealer_hand)    
    
        show_all(player_hand,dealer_hand)
        
        # end of game scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand,dealer_hand,player_chips)

        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand,dealer_hand,player_chips)

        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand,dealer_hand,player_chips)

        else:
            push(player_hand,dealer_hand) 
    
    # print results 
    print("\nPlayer's winnings stand at",player_chips.total)
    
    # loop or break
    new_game = input("Would you like to play another hand? Enter 'y' or 'n' ")
    
    if new_game[0].lower()=='y':
        playing=True
        continue
    else:
        print("Thanks for playing!")
        break